#**Importando pacotes**

In [1]:
!pip install pyproj

In [2]:
import pandas as pd

In [3]:
from pyproj import Proj, transform

In [4]:
from bokeh.plotting import output_notebook, output_file, save, show, figure
from bokeh.tile_providers import Vendors, get_provider
from bokeh.models import Circle,ColumnDataSource,HoverTool,CustomJS, WheelZoomTool

# **Carregando dados**

In [5]:
data = pd.read_csv("abretecodigo.csv").rename(columns={"Source ID":"id"})

In [6]:
data.head(5)

,id,title,original title,date,date circa,creator,creator info,place,city,state,...,collection,description,license,latitude,longitude,altitude,heading,tilt,fov,media
0,0072430cx098-08,Teatro Municipal,NaN,NaN,1905/1915,Marc Ferrez,"Nascido em 1843 no Rio de Janeiro, Marc Ferrez...",Centro; Cinelândia,Rio de Janeiro,RJ,...,Coleção Gilberto Ferrez,"Vista do Teatro Municipal, vendo-se a Escola N...",NaN,-22.910046,-43.175833,9.70,-23.75,96.24,50.0,https://rioiconography.sfo2.digitaloceanspaces...
1,0072430cx097-12,"Museu Nacional, antigo Palácio Imperial de São...",NaN,NaN,1879/1889,Marc Ferrez,"Nascido em 1843 no Rio de Janeiro, Marc Ferrez...",São Cristóvão; Quinta da Boa Vista,Rio de Janeiro,RJ,...,Coleção Gilberto Ferrez,"Ao longo do tempo, o Paço de São Cristóvão, qu...",NaN,-22.903364,-43.223602,63.39,-136.47,91.38,41.6,https://rioiconography.sfo2.digitaloceanspaces...
2,0072430cx097-10,Santa Casa de Misericórdia,NaN,NaN,1875/1885,Marc Ferrez,"Nascido em 1843 no Rio de Janeiro, Marc Ferrez...",Centro; Rua Santa Luzia,Rio de Janeiro,RJ,...,Coleção Gilberto Ferrez,Prédio da Santa Casa de Misericórdia no centro...,NaN,-22.907723,-43.170320,9.74,-112.60,91.92,50.0,https://rioiconography.sfo2.digitaloceanspaces...
3,0072430cx097-09,Imprensa Nacional,NaN,NaN,1889/1899,Marc Ferrez,"Nascido em 1843 no Rio de Janeiro, Marc Ferrez...",Centro; Rua Treze de Maio,Rio de Janeiro,RJ,...,Coleção Gilberto Ferrez,"Prédio da Imprensa Nacional, ao fundo o morro ...",NaN,-22.906498,-43.177990,18.85,-169.39,87.90,39.8,https://rioiconography.sfo2.digitaloceanspaces...
4,0072430cx097-06,Imprensa Nacional,NaN,NaN,1889/1899,Marc Ferrez,"Nascido em 1843 no Rio de Janeiro, Marc Ferrez...",Centro; Rua Treze de Maio,Rio de Janeiro,RJ,...,Coleção Gilberto Ferrez,"Prédio da Imprensa Nacional, ao fundo o morro ...",NaN,-22.906569,-43.178173,25.21,-172.23,86.06,45.4,https://rioiconography.sfo2.digitaloceanspaces...


# **Convertendo projeções**

In [7]:
def transform_proj(row):
    """
    Transforma WGS84(epsg:4326) para WEBMERCATOR(epsg:3857)
    """

    try:
        proj_in = Proj("epsg:4326")
        proj_out = Proj("epsg:3857")
        x1, y1 = row["latitude"], row["longitude"]
        x2, y2 = transform(proj_in, proj_out, x1, y1)
        return pd.Series([x2, y2])

    except Exception as e:
        print(str(e))

In [8]:
data[["lat_conv","lng_conv"]] = data.apply(transform_proj, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  # Re

# **Visualização**

## **Construindo a primeira layer: mapa**

In [9]:
mapa_base = figure(
    x_axis_type="mercator",
    y_axis_type="mercator",
    plot_width=1400,
    plot_height=900,
    toolbar_location=None,
)

tile_provider = get_provider(Vendors.CARTODBPOSITRON_RETINA)
mapa_base.add_tile(tile_provider)

TileRenderer(id='1042', ...)

## **Filtro dos dados e interação**

In [10]:
source = ColumnDataSource(data)

In [11]:
TOOLTIPS = """
        <div style="margin: 5px; width: 300px" >
        <img
            src="@media" alt="@media" height=200
            style="margin: 0px;"
            border="2"
            ></img>
            <p style='font-size: 13px; font-weight: bold;'>@title</p>
            <p style='font-size: 10px;'>@place</p>
            <p style='font-size: 10px; font-weight: light; font-style: italic;'>@creator</p>
        
        </div>
    """
callback = CustomJS(
    code="""
         var tooltips = document.getElementsByClassName("bk-tooltip");
         for (var i = 0, len = tooltips.length; i < len; i ++) {
              tooltips[i].style.top = ""; 
              tooltips[i].style.right = "";
              tooltips[i].style.bottom = "0px";
              tooltips[i].style.left = "0px";
              }
          """
)

## **Construindo a segunda layer: objetos**

In [12]:
point = mapa_base.circle(
        x="lat_conv",
        y="lng_conv",
        source=data,
        size=7,
        fill_color="orange",
        fill_alpha=0.5,
        line_color="dimgray",
)

## **Adicionando interações**

In [13]:
hover = HoverTool(
        renderers=[point],
        tooltips=TOOLTIPS,
        mode="mouse",
        show_arrow=False,
        callback=callback,
)
mapa_base.add_tools(hover)
mapa_base.toolbar.active_scroll = mapa_base.select_one(WheelZoomTool)

# **Resultado**

## **Visualização em notebook**

In [14]:
output_notebook()
show(mapa_base)

## **Visualização em HTML**

In [15]:
output_file(filename="mapa_fotografias_rj.html", title="Fotografias do Rio de Janeiro entre os séculos XIX e XX")
save(mapa_base)

'/content/mapa_fotografias_rj.html'